# calculo de distancia de las estaciones al mar
### Diana Jaimes- consultora (dic 2022)

In [1]:
from random import uniform
from concurrent.futures import ThreadPoolExecutor
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import geopandas as gpd
import xarray as xr
import geopandas as gpd
from shapely.geometry import Point, box
import cartopy

## Se obtiene las coordenadas de las estaciones de ICC

In [2]:
general_info=pd.read_excel('información_general_REDMET.xlsx')
general_info = general_info[['Codigo', 'Latitud', 'Longitud']]
general_info.rename(columns = {'Codigo':'codigo', 'Latitud':'latitud', 'Longitud':'longitud'}, inplace = True)
general_info = general_info.head(38).reset_index(drop=True)


,codigo,latitud,longitud
0,CEN-CEN,14.330962,-91.055468
1,LUT-TEH,14.168625,-91.103443
2,PAN-BAL,14.281468,-91.003744
3,TBU-PUY,14.261557,-91.25991
4,MAG-SAV,13.995364,-91.200961
5,SAA-AMA,14.066847,-90.774593
6,SDT-TRI,14.153762,-90.844006
7,TBU-IRL,14.145889,-91.426867
8,LUT-BON,14.078341,-91.187235
9,MAG-BOU,14.11769,-90.933352


In [11]:
rows = general_info.longitud.to_list()
cols = general_info.latitud.to_list()

# put data into a dataset
general_info_xr = xr.Dataset(
    coords=dict(
        lon=(["longitud"], rows),
        lat=(["latitud"], cols),
    ),
    attrs=dict(description="coordinates of different stations"),
)
general_info_xr

<xarray.Dataset>
Dimensions:  (longitud: 38, latitud: 38)
Coordinates:
    lon      (longitud) float64 -91.06 -91.1 -91.0 ... -88.8 -90.97 -91.02
    lat      (latitud) float64 14.33 14.17 14.28 14.26 ... 13.4 14.74 14.09
Dimensions without coordinates: longitud, latitud
Data variables:
    *empty*
Attributes:
    description:  coordinates of different stations

In [12]:
def coastline_coord(country_name):
    print('data frame en geopandas con info de poligonos para diferentes lugares')
    world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
    #single geom for country
    country = world[world["name"]==country_name].dissolve(by='name').iloc[0].geometry
    #single geom for the coastline
    c = cartopy.io.shapereader.natural_earth(resolution='50m', category='physical', name='coastline')
    c = gpd.read_file(c)
    c.crs = 'EPSG:4326'
    print('Get coastline...')
    coastline = gpd.clip(c.to_crs('EPSG:4326'), country.buffer(0.25)).iloc[0].geometry
    return coastline


In [13]:
coastline = coastline_coord('Guatemala')
def compute_distance(point):
   point['dist_to_coastline'] = point['geometry'].distance(coastline)
   return point

data frame en geopandas con info de poligonos para diferentes lugares
Get coastline...


/home/diana/.local/lib/python3.10/site-packages/geopandas/tools/clip.py:67: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  clipped.loc[


In [14]:
print('Group lat/lon points...')
stations = []
i = 0
for ilat in general_info_xr['lat']:
    stations.append({'id':i, 'geometry':Point(general_info_xr['lon'][i],ilat)})
    i+=1
stations

Group lat/lon points...


[{'id': 0, 'geometry': <shapely.geometry.point.Point at 0x7fa43327f850>},
 {'id': 1, 'geometry': <shapely.geometry.point.Point at 0x7fa433998730>},
 {'id': 2, 'geometry': <shapely.geometry.point.Point at 0x7fa438a81960>},
 {'id': 3, 'geometry': <shapely.geometry.point.Point at 0x7fa43399b190>},
 {'id': 4, 'geometry': <shapely.geometry.point.Point at 0x7fa4332aa200>},
 {'id': 5, 'geometry': <shapely.geometry.point.Point at 0x7fa43327f910>},
 {'id': 6, 'geometry': <shapely.geometry.point.Point at 0x7fa4332ab8e0>},
 {'id': 7, 'geometry': <shapely.geometry.point.Point at 0x7fa4332ab250>},
 {'id': 8, 'geometry': <shapely.geometry.point.Point at 0x7fa4332a8730>},
 {'id': 9, 'geometry': <shapely.geometry.point.Point at 0x7fa4332abf10>},
 {'id': 10, 'geometry': <shapely.geometry.point.Point at 0x7fa4332aaf20>},
 {'id': 11, 'geometry': <shapely.geometry.point.Point at 0x7fa4332a91b0>},
 {'id': 12, 'geometry': <shapely.geometry.point.Point at 0x7fa4332a8ac0>},
 {'id': 13, 'geometry': <shapely.ge

In [15]:
def coastline_distance(points):
    print('Computing distances...')
    with ThreadPoolExecutor(max_workers=4) as tpe:
        result = list(tqdm(tpe.map(compute_distance, points), desc="computing distances", total=len(points)))
    gdf = gpd.GeoDataFrame.from_records(result)
    print('Convert to xarray...')
    lon = gdf['geometry'].x
    lat = gdf['geometry'].y
    df1 = pd.DataFrame(gdf)
    df1['latitud'] = lat
    df1['longitud'] = lon
    df1 = df1.drop(columns=['id','geometry'])
    return df1


In [21]:
coastline_distance(stations).merge(general_info, how='inner', on=['longitud', 'latitud']).to_excel('distance_costline.xlsx')

Computing distances...


computing distances:   0%|          | 0/38 [00:00<?, ?it/s]

Convert to xarray...


,dist_to_coastline,latitud,longitud,codigo
0,0.404794,14.330962,-91.055468,CEN-CEN
1,0.242764,14.168625,-91.103443,LUT-TEH
2,0.354973,14.281468,-91.003744,PAN-BAL
3,0.292958,14.261557,-91.25991,TBU-PUY
4,0.052434,13.995364,-91.200961,MAG-SAV
5,0.138903,14.066847,-90.774593,SAA-AMA
6,0.226257,14.153762,-90.844006,SDT-TRI
7,0.119548,14.145889,-91.426867,TBU-IRL
8,0.136045,14.078341,-91.187235,LUT-BON
9,0.190752,14.11769,-90.933352,MAG-BOU


In [20]:
general_info_xr

<xarray.Dataset>
Dimensions:  (longitud: 38, latitud: 38)
Coordinates:
    lon      (longitud) float64 -91.06 -91.1 -91.0 ... -88.8 -90.97 -91.02
    lat      (latitud) float64 14.33 14.17 14.28 14.26 ... 13.4 14.74 14.09
Dimensions without coordinates: longitud, latitud
Data variables:
    *empty*
Attributes:
    description:  coordinates of different stations

In [ ]:
.merge(general_info, how='inner', on=['longitud', 'latitud'])